## Load the libraries

In [ ]:
#fuser -v /dev/nvidia*
#sudo pkill -f ipykernel

In [ ]:
%load_ext autoreload
%autoreload 2
import data_processor
import time
import os
import io
import shutil
import numpy as np
from PIL import Image
import tensorflow as tf
import argparse
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint, Callback

from keras.utils.training_utils import multi_gpu_model
from keras.utils import plot_model
from keras.optimizers import Adam, SGD
import keras.backend as K

from adamw import AdamW
from keras.models import load_model, model_from_json

from model import EAST_model
from losses import dice_loss, rbox_loss
import data_processor
from data_processor import restore_rectangle
import lanms
import random
import cv2
import gc 
from for_train import *

parser = argparse.ArgumentParser()
parser.add_argument('--input_size', type=int, default=512) # input size for training of the network
parser.add_argument('--batch_size', type=int, default=8) # batch size for training
parser.add_argument('--nb_workers', type=int, default=4) # number of processes to spin up when using process based threading, as defined in https://keras.io/models/model/#fit_generator
parser.add_argument('--init_learning_rate', type=float, default=0.01) # initial learning rate
parser.add_argument('--lr_decay_rate', type=float, default=0.94) # decay rate for the learning rate
parser.add_argument('--lr_decay_steps', type=int, default=130) # number of steps after which the learning rate is decayed by decay rate
parser.add_argument('--max_epochs', type=int, default=800) # maximum number of epochs
parser.add_argument('--gpu_list', type=str, default='0') # list of gpus to use
parser.add_argument('--checkpoint_path', type=str, default='tmp/east_resnet_50_rbox') # path to a directory to save model checkpoints during training
parser.add_argument('--save_checkpoint_epochs', type=int, default=5) # period at which checkpoints are saved (defaults to every 10 epochs)
parser.add_argument('--training_data_path', type=str, default='../data/train/augmented_images') # path to training data
parser.add_argument('--validation_data_path', type=str, default='../data/val/augmented_images') # path to validation data
parser.add_argument('--max_image_large_side', type=int, default=1280) # maximum size of the large side of a training image before cropping a patch for training
parser.add_argument('--max_text_size', type=int, default=800) # maximum size of a text instance in an image; image resized if this limit is exceeded
parser.add_argument('--min_text_size', type=int, default=10) # minimum size of a text instance; if smaller, then it is ignored during training
parser.add_argument('--min_crop_side_ratio', type=float, default=0.1) # the minimum ratio of min(H, W), the smaller side of the image, when taking a random crop from thee input image
parser.add_argument('--geometry', type=str, default='RBOX') # geometry type to be used; only RBOX is implemented now, but the original paper also uses QUAD
parser.add_argument('--suppress_warnings_and_error_messages', type=bool, default=True) # whether to show error messages and warnings during training (some error messages during training are expected to appear because of the way patches for training are created)
FLAGS, unknown = parser.parse_known_args()
gpus = list(range(len(FLAGS.gpu_list.split(','))))

In [ ]:
RESIZE_FACTOR = 2

## Split in Train and validation

In [ ]:
TRAIN_FOLDER = "../data/train/augmented_images/"
VAL_FOLDER = "../data/val/augmented_images/"

In [ ]:
val_list_orig = [f[:-4] for f in os.listdir(VAL_FOLDER) if f[-4:] ==".png"]
for i in range(len(val_list_orig)):
    shutil.move(VAL_FOLDER + val_list_orig[i] + ".png", TRAIN_FOLDER + 
                val_list_orig[i] + ".png")
    shutil.move(VAL_FOLDER + val_list_orig[i] + ".txt", TRAIN_FOLDER + 
                val_list_orig[i] + ".txt")

del val_list_orig
    
f_list = [f[:-4] for f in os.listdir(TRAIN_FOLDER) if f[-4:] ==".png"]
val_list = random.sample(f_list, 50)

for i in range(len(val_list)):
    shutil.move(TRAIN_FOLDER + val_list[i] + ".png", VAL_FOLDER +
                val_list[i] + ".png")
    shutil.move(TRAIN_FOLDER + val_list[i] + ".txt", VAL_FOLDER +
                val_list[i] + ".txt")

f_list_train = [f[:-4] for f in os.listdir(TRAIN_FOLDER) if f[-4:] ==".png"]
f_list_val = [f[:-4] for f in os.listdir(VAL_FOLDER) if f[-4:] ==".png"]

print(len(f_list_train))
print(len(f_list_val))

del f_list
del val_list
del f_list_train
del f_list_val

gc.collect()

## Create Data Generators

In [ ]:
val_data = data_processor.load_data(FLAGS)

In [ ]:
train_data_generator = data_processor.generator(FLAGS)
train_samples_count = data_processor.count_samples(FLAGS)
train_samples_count

## Create Model

In [ ]:
east = EAST_model(FLAGS.input_size)
parallel_model = east.model

## Load weights (if available)

In [ ]:
parallel_model.load_weights(FLAGS.checkpoint_path + "/model-90.h5")

In [ ]:
def lr_decay(epoch):
    return FLAGS.init_learning_rate * np.power(FLAGS.lr_decay_rate, epoch // FLAGS.lr_decay_steps)

In [ ]:
model_json = east.model.to_json()
with open(FLAGS.checkpoint_path + '/model.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = FLAGS.gpu_list

# check if checkpoint path exists
if not os.path.exists(FLAGS.checkpoint_path):
    os.mkdir(FLAGS.checkpoint_path)


In [ ]:
score_map_loss_weight = K.variable(0.01, name='score_map_loss_weight')
small_text_weight = K.variable(0., name='small_text_weight')

lr_scheduler = LearningRateScheduler(lr_decay)
ckpt = CustomModelCheckpoint(model=parallel_model, path=FLAGS.checkpoint_path + '/model-{epoch:02d}.h5', period=FLAGS.save_checkpoint_epochs, save_weights_only=True)
tb = CustomTensorBoard(log_dir=FLAGS.checkpoint_path + '/train', score_map_loss_weight=score_map_loss_weight, small_text_weight=small_text_weight, data_generator=train_data_generator, write_graph=True)
small_text_weight_callback = SmallTextWeight(small_text_weight)
validation_evaluator = ValidationEvaluator(val_data, validation_log_dir=FLAGS.checkpoint_path + '/val')
callbacks = [lr_scheduler,ckpt, tb, small_text_weight_callback, validation_evaluator]

## Train the Model

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)



In [ ]:
#for i in  range(10):
    #NB_LAYERS_TO_TRAIN = 5 + (5*i)    
    #INIT = 20 + (i*10)
NB_LAYERS_TO_TRAIN = 100
INIT = 25
for layer in parallel_model.layers[:-NB_LAYERS_TO_TRAIN]:
    layer.trainable = False
for layer in parallel_model.layers[-NB_LAYERS_TO_TRAIN:]:
    layer.trainable = True

opt = AdamW(FLAGS.init_learning_rate)

parallel_model.compile(loss=[dice_loss(east.overly_small_text_region_training_mask, 
                                       east.text_region_boundary_training_mask, score_map_loss_weight, 
                                       small_text_weight), 
                             rbox_loss(east.overly_small_text_region_training_mask, 
                                       east.text_region_boundary_training_mask, small_text_weight, 
                                       east.target_score_map)], loss_weights=[1., 1.], 
                       optimizer=opt)

history = parallel_model.fit_generator(train_data_generator, epochs=800,
                                       initial_epoch = 0,
                                       steps_per_epoch=train_samples_count/FLAGS.batch_size,
                                       use_multiprocessing=True, 
                                       max_queue_size=1, callbacks=callbacks, verbose=1, 
                                       workers=FLAGS.nb_workers)

## Export the model

In [ ]:
shutil.copyfile(FLAGS.checkpoint_path + '/model.json', "../model_east.json")
shutil.copyfile(FLAGS.checkpoint_path + "/model-90.h5", "../weights_east.h5")